### Spark Join Transformation

### INNER JOIN

In [0]:
%run "../includes/configuration"

In [0]:
movie_df = spark.read.parquet(f"{silver_folder_path}/movies").filter("year_release_date  ==2007")

In [0]:
production_country = spark.read.parquet(f"{silver_folder_path}/productions_countries")

In [0]:
country_df = spark.read.parquet(f"{silver_folder_path}/countries")

In [0]:
display(movie_df)

In [0]:
display(production_country)

In [0]:
display(country_df)

In [0]:
#Tenemos que tener en cuenta que movie_df, es una tabla que se encuentra en a posicion izquierda
#production_country, es una tabla que se encuentra en la posicion derecha
#Por defecto el tipo de Join que utiliza es INNER JOIN

#join recibe 3 parametros, el primer parametro es la tabla con la que voy hacer el join (izquierda)
# El segundo parametro son las columnas que voy a unir, en este caso movie_id 
# El tercer parametro es el tipo de join que se va a ejectuar
movie_production_country_df  = movie_df.join(production_country,
                                             movie_df.movie_id == production_country.movie_id,"inner")

In [0]:
# Al usar .select, indicamos las columnas que ocupamos
movie_production_country_df  = movie_df.join(production_country,
                                             movie_df.movie_id == production_country.movie_id,"inner") \
                                             .select(movie_df.title, movie_df.budget, production_country.country_id)

In [0]:
movie_country_df = movie_production_country_df.join(country_df, 
                                                    movie_production_country_df.country_id == country_df.
                                                    country_id, 
                                                    "inner")

In [0]:
#Tenemos la informacion de los paises donde se han grabado las peliculas, pais, presupuesto y titulo de la pelicula
movie_country_df = movie_production_country_df.join(country_df, 
                                                    movie_production_country_df.country_id == country_df.
                                                    country_id, 
                                                    "inner") \
                    .select(movie_production_country_df.title, movie_production_country_df.budget, country_df.country_name)
                                                        

In [0]:
display(movie_country_df)

### Outer Join

##### Left Outer Join

In [0]:
# Outer Join en Spark combina 2 DataFrames o tablas y devuelve todas las filas de ambas, incluso si no hay coincidencias entre ellas. Si una fila no tiene coincidencias en otro DataFrame o tabla. Las columnas correspondientes tendran valores NULL

# movie_df => se encuentra en el lado derecho
# production_country => se encuentra en el lado izquierdo
movie_production_country_df  = movie_df.join(production_country,
                                             movie_df.movie_id == production_country.movie_id,"left") \
                                             .select(movie_df.title, movie_df.budget, production_country.country_id)

In [0]:
movie_country_df = movie_production_country_df.join(country_df, 
                                                    movie_production_country_df.country_id == country_df.
                                                    country_id, 
                                                    "left") \
                    .select(movie_production_country_df.title, movie_production_country_df.budget, country_df.country_name)

#### Right Outer Join

In [0]:
# movie_df => se encuentra en el lado izquierdo
# production_country => se encuentra en el lado derecho
movie_production_country_df  = movie_df.join(production_country,
                                             movie_df.movie_id == production_country.movie_id,"right") \
                                             .select(movie_df.title, movie_df.budget, production_country.country_id)

In [0]:
movie_country_df = movie_production_country_df.join(country_df, 
                                                    movie_production_country_df.country_id == country_df.
                                                    country_id, 
                                                    "right") \
                    .select(movie_production_country_df.title, movie_production_country_df.budget, country_df.country_name)

#### Full Outer Join

In [0]:
movie_production_country_df  = movie_df.join(production_country,
                                             movie_df.movie_id == production_country.movie_id,"full") \
                                             .select(movie_df.title, movie_df.budget, production_country.country_id)

In [0]:
movie_country_df = movie_production_country_df.join(country_df, 
                                                    movie_production_country_df.country_id == country_df.
                                                    country_id, 
                                                    "full") \
                    .select(movie_production_country_df.title, movie_production_country_df.budget, country_df.country_name)

#### Semi Join

In [0]:
movie_production_country_df  = movie_df.join(production_country,
                                             movie_df.movie_id == production_country.movie_id,"left") \
                                             .select(movie_df.title, movie_df.budget, production_country.country_id)

In [0]:
#Semi Join solo me devueve columnas del lado izquerdo, no podemos seleccionar columnas del lado derecho
# movie_production_country_df = lado izquierdo
# country_df = lado derecho
movie_country_df = movie_production_country_df.join(country_df, 
                                                    movie_production_country_df.country_id == country_df.
                                                    country_id, 
                                                    "semi") \
                    .select(movie_production_country_df.title, movie_production_country_df.budget)

#### Anti Join

In [0]:
movie_production_country_df  = movie_df.join(production_country,
                                             movie_df.movie_id == production_country.movie_id,"left") \
                                             .select(movie_df.title, movie_df.budget, production_country.country_id)

In [0]:
display(movie_production_country_df)

In [0]:
# Devuelve 8 valores, estos valores son los que no han hecho coincidencia con el lado derecho del DataFrame
movie_country_df = movie_production_country_df.join(country_df, 
                                                    movie_production_country_df.country_id == country_df.
                                                    country_id, 
                                                    "anti") \
                    .select(movie_production_country_df.title, movie_production_country_df.budget)

#### Cross Join

In [0]:
# Realiza el producto cartesiano
movie_country_df = movie_df.crossJoin(country_df) 

In [0]:
display(movie_country_df.count())

In [0]:
#Producto cartesiano delos datos, da el mismo resultado del count del crossjoin de la consulta anterior
int(movie_df.count()) * int(country_df.count())

##### Se requiere obtener informacion de las "peliculas" del año 2000 en adelante ordenado de manera descendete por la fecha de lanzamiento, se debe considerar su respectivo "genero" y su "idioma" que se encuentra en dicha pelicula.

1. De la pelicula se requiere mostrar el "titulo, el tiempo de duracion, fecha lanzamiento y el voto promedio"
2. Del idioma se requiere mostrar el  nombre del idioma
3. Del genero mostrar el nombre del genero